In [1]:
import sys
import os

# Agregar la ruta absoluta del directorio 'img' al sys.path
utils_path = os.path.abspath(os.path.join(os.getcwd(), "src", "img"))

## MEMORIA - EDA MONITORIZACIÓN DE LA DEFORESTACIÓN DE LOS BOSQUES AMAZÓNICOS. CASO DE ESTUDIO EN PERÚ

### **1. CONTEXTO**

* La **deforestación**, además de la pérdida de biodiversidad y servicios ecosistémicos locales y regionales, supone **una de las principales fuentes de emisión de gases de efecto invernadero (GEI)**. Desde 1970, las emisiones acumuladas de $CO_2$ procedentes de la deforestación y cambio de usos del suelo han incrementado un 40\%; y en 2010 ya suponían el 24\% del total de las emisiones de GEI; más que las emisiones procedentes del sector industrial (21\%) o del transporte (14\%)

* **Si la deforestación tropical fuera un país, tendría la tercera mayor huella de carbono del mundo**. Según datos e imágenes de Global Forest Watch, analizados en una investigación recientemente publicada por la revista Nature, la pérdida de cobertura arbórea tropical provocó un promedio de emisiones anuales equivalentes a 5,3 gigatoneladas entre 2001 y 2019. Esta cifra la sitúa en tercer lugar, después de China y Estados Unidos, si se excluyen los efectos del cambio de uso del suelo y la silvicultura.

<div style="text-align: center;">
    <img src="./src/img/deforestacion_emisiones.jpeg" alt="Emisiones de CO₂ por deforestación" style="width:25%;"/>
</div>


* En el contexto actual de cambio climático, y sin una reducción eficiente y sostenida de las emisiones de GEI, es muy importante contar con **sistemas automáticos de monitorización** que faciliten y fomenten la implementación de **programas para la conservación de bosques**, como el programa [REED+](https://unfccc.int/topics/land-use/workstreams/redd/what-is-redd) de las Naciones Unidas o los proyectos de captura y absorción de $CO_2$ en bosques ([carbon farming](https://eu-cap-network.ec.europa.eu/publications/carbon-farming-projects-brochure_en)). Estos sistemas deben ser transparentes, sólidos, robustos y fiables para evitar prácticas de [green washing](https://blogs.lse.ac.uk/internationaldevelopment/2023/01/26/the-verra-scandal-explained-why-avoided-deforestation-credits-are-hazardous/?utm_source=chatgpt.com) que cuestionan la implantación de este tipo de soluciones climáticas. En este sentido, los **sistemas basados en datos de teledetección (imágenes satelitales ópticas, radas, LiDAR, etc) y modelos de inteligencia artificia**l para la identificación de patrones de pérdida o ganancia de biomasa forestal, y estimación de balances netos de $CO_2$, se consideran una herramienta con gran potencial ([Monitoring of forests through remote sensing](https://op.europa.eu/en/publication-detail/-/publication/38567f41-288b-11eb-9d7e-01aa75ed71a1)).

#### CASO DE ESTUDIO

* **Perú contiene el 16% de los bosques amazónicos del mundo**. Dentro del país, una de las regiones más afectadas por la deforestación es el departamento de Ucayali. En esta región el Gobierno de Perú a reportado una **pérdida de más de 540.000 ha en el periodo 2001 - 2021**. Se ha seleccionado un área de estudio en esta región, en la provincia de Padre Abad, en la que la expansión de las actividades ganaderas y agroindustriales han provocado importantes pérdidas de cobertura vegetal. En la siguiente imagen se muestra el área de estudio. En color rojo se resaltan las áreas afectadas por la deforestación para el periodo 2001-2021 según datos del [Gobierno de Perú](https://geobosques.minam.gob.pe/geobosque/view/descargas.php?122345gxxe345w34gg).

<div style="text-align: center;">
    <img src="./src/img/Figura1_a.png" alt="ROI" style="width:25%;"/>
</div>




### **2. HIPÓTESIS**

1. ¿Que patrón de deforestación se identifica en el caso de estudio? 

2. ¿Se puede identificar y caracterizar, de manera robusta, la pérdida de cobertura vegetal a partir de la información proporcionada por modelos de ajuste y segementación de series temporales de imágenes satelitales como [LandTrendr](https://geotrendr.ceoas.oregonstate.edu/landtrendr/) o [CCDC](https://gee-ccdc-tools.readthedocs.io/en/latest/background.html)?

3. ¿Porían servir estos algoritmos como base para el desarrollo de sistemas robustos de monitoreo, reporte y verificación (MRV) de emisiones y captura de $CO_2$? 

4. ¿Que ventaja ofrece el uso de estos datos frente a otro tipo de información espectral?


### **3. DATOS**

#### RECOLECCIÓN DE DATOS

1. ***Dataset con la información espectral***: Datos obtenidos al aplicar el algoritmo LandTrendr para la segmentación de una serie temporal de las imágenes de alta resolución Landsat (30m) desde el año 1985 al año 2022, en la plataforma *Google Earth Engine*. El modelo LandTrendr ajusta los valores espectrales de la serie temporal de imágenes en un modelo de segmentos lineales que capturan tanto los cambios bruscos de la señal espectral (*Spectral Index value*), como las tendencias graduales, descartanto aquellos valores que son producto de ruido de fondo (nubosidad, ruido electrónico o digital del sensor, etc). En la siguiente imagen se muestra el modelo conceptual. Se puede consultar más información sobre el modelo y su implementación en GEE en este enlace: [LT-GEE Guide](https://emapr.github.io/LT-GEE/).

<div style="text-align: center;">
    <img src="./src/img/Figura3_ModConceptual_LandTrend.png" alt="Emisiones de CO₂ por deforestación" style="width:30%;"/>
</div>

- Para el estudio de deforestación se ha seleccionado, para cada píxel, el segmento que representa pérdida de valor espectral (pendiente negativa) de mayor magnitud (mayor diferencia del valor espectral entre los vértices del segmento), entre los años 2001 y 2021. El resultado se ha exportado en un archivo imagen de formato Geotiff desde GEE, con 6 bandas. En cada banda se guardan los siguientes atributos que caracterizan el segmento:

    - **Banda 1 - Año del cambio o perturbación (yod)**: Año del cambio que representa el segmento.
    - **Banda 2 - Magnitud**: Diferencia del valor espectral de los dos vértices del segmento.
    - **Banda 3 - Duración**: Diferencia de los años de los dos vértices del segmento
    - **Banda 4 - Valor espectral previo al cambio**: valor del índice espectral en el primer vértice del segmento (preval)
    - **Banda 5 - Tasa**: Calculada como $magnitud/duración$
    - **Banda 6 - ratio DSNR**: ratio que mide la proporcion de señal espectral respecto del ruido de fondo
    
- El valor espectral sobre el que se ha realizado la segmentación ha sido el **índice TCW (*Tasseled Cap Wetness*)** que es muy sensible a los cambios en la cobertura vegetal. A continuación se muestra su formula de cálculo:
    
$$
\small
TCW = 0.0315 \cdot B + 0.2021 \cdot G + 0.3102 \cdot R + 0.1594 \cdot NIR - 0.6806 \cdot SWIR1 - 0.6109 \cdot SWIR2
$$

$$
\small
\begin{aligned}
&NIR: \text{ banda infrarrojo cercano} \\
&SWIR1: \text{ banda infrarrojo de onda corta 1} \\
&SWIR2: \text{ banda infrarrojo de onda corta 2} \\
&B: \text{ banda azul} \\
&G: \text{ banda verde} \\
&R: \text{ banda rojo} \\
\end{aligned}
$$



2. ***Dataset de referencia***: Para identificar las áreas deforestadas de las no deforestadas se ha generado una máscara booleana, en formato Geotiff, para el periodo 2001-2021, a partir de los datos del producto denominado *Hansen Global Forest Change v1.11 (2000-2023)*, disponible en el catálogo de datos de GEE ([GFC](https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2023_v1_11)). Este producto, identifica a nivel global, las pérdidas y ganancias de cobertura forestal.


#### LIMPIEZA Y PREPARACIÓN DE DATOS

* Visualización de la información espectral contenida en el geotiff, y creación de un dataframe a partir de la información contenida en cada una de las bandas. Se ha creado una columna para la inforamción contenida en cada una de las bandas

* Creación de un dataframe, con la columna 'clasificacion' a partir de la imagen con los datos de referencia (un única banda)

* Unión de los dos dataframes a partir de la información de localización de los píxeles en los ejes x e y

* Comprobación de coincidencia de coordenadas geográficas de cada píxel en los dos dataframe, para validar que las dos imágenes de dnode proceden los datos están co-registradas (alineadas geográficamente).

* El % de NaN en el df resultante es muy alto porque corresponden con los píxeles de la imagen que están fuera del área de estudio (valores enmascarados) y que al exportar la imagen con los resultados desde GEE, como tipo float, se registran como valores NaN. Se elimiman porque corresponde con registros que están fuera del área de estudio. Después de eliminar los NaN se cuenta con más de millón y medio de registros para el análisis

* No se identifican valores duplicados

* Se renombran las columnas para facilitar el acceso a las variables

* Se normalizan, a valores entre 0 y 1, los valores de las variables magnitud, preval y tasa para que facilitar la comparabilidad entre variables

* Se convierten los valores de yod, duración y clasificacion a tipo int

* Se crea una columna, tipo str, para los valores de la columna categórica clasificación, para facilitar la comparación posterior entre grupos

* Finalmente se analiza la cardinalidad de las variables.

En la siguiente tabla se resumen las principales características de las variables del dataframe final ue se usa como base del análisis.


##### TABLA DE VARIABLES

|Columna/Variable|Descripción|Tipo_de_Variable|Rol en el EDA|Nota|
|-|-|-|-|-|
|x|Coordenada x de localización del píxel en la imagen|Numérica discreta|Define posición en x del píxel dentro de la imagen. Para que se pueda considerar como índice de los registros, habría que combinarla con los valores de y||
|y|Coordenada y de localización del píxel en la imagen|Numérica discreta|Define posición en y del píxel dentro de la imagen. Para que se pueda considerar como índice de los registros, habría que combinarla con los valores de x||
|long|Coordenada longitud de localización del píxel en la imagen|Numérica discreta|Define posición del píxel con coordenadas geográficas de longitud dentro del ROI||
|lat|Coordenada latitud de localización del píxel en la imagen|Numérica discreta|Define posición del píxel con coordenadas geográficas de latitud dentro del ROI||
|clasificación|Variable con los datos de referencia que indica si ha habido deforestación en el píxel|Categórica/Binaria|Variable directora del análisis|valor 1 = no deforestación, valor 2 = deforestación|
|mag_norm|Variable normalizada que representa la magnitud del cambio (negativo) en la señal espectral del píxel. La señal espectral se mide como el índice TCW que es sensible a los cambios en coberturas vegetales|Numérica continua|Esencial para identificar y caracterizar los cambios que podrían estar relacionados con eventos de deforestación||
|yod|Variable que indica el año en el que se ha producido un cambio negativo en la señal espectral|Numérica discreta|Importante para identificar los patrones temporales de deforestación del ROI (área de estudio)||
|dur|Variable que indica la duración del segemento que define al cambio negativo en la señal espectral|Numérica discreta|Esencial para identificar y caracterizar los cambios que podrían estar relacionados con eventos de deforestación (eventos de corta duración)||
|rate_norm|Variable normalizada que indica la tasa de cambio del segeento que se ajusta al cambio negativo en la señal espectral. Se calcula como mag/dur|Numérica continua|Importante para caracterizar los eventos de pérdida de cobertura vegetal y posibles causas de los mismos. Ej: Un evento de deforestación por incencio generalmente tiene tasas más altas que eventos de degradación de la cobertura vegetal graduales generados por tala ilegal||
|preval_norm|Variable normalizada que representa el valor de la señal espectral antes de producirse el cambio|Numérica continua|Indicador del verdor previo al evento de pérdida cobertura vegetal. Permite conocer el estado de la cobertura vegetal previa al evento||
|dsnr|Variable que representa la proporción de señal espectral frente al ruido espectral presente en la señal|Numérica continua|Importante para caracterizar la fiabilidad del modelo de segmentación. Valores bajos indican que la segmentación está fuertemente influenciada por el ruido espectral vs. cambios de señal espectral relacionados con eventos de pérdida de cobertura vegetal||


Para más detalle sobre los datos y el proceso de ETL, consultar el archivo [main.ipynb](./main.ipynb)

### **4. ANÁLISIS REALIZADO Y RESULTADOS**


### **5. CONCLUSIONES Y RECOMENDACIONES BASADAS EN LOS HALLAZGOS**
